In [4]:
import numpy as np
import pickle as pkl
from pprint import pprint
import os
import matplotlib
import matplotlib.pyplot as plt
plt.ion()


path_source_files = '/Users/tianyichen/Desktop/Research /PhDresearch/Hopkins_Organoid'
# Get list of all pickle files in the target directory
filenames = os.listdir(path_source_files)
filenames = [fn for fn in filenames if fn.endswith('.pkl')]
sizes = []
for f in filenames:
    sizes.append(os.stat(os.path.join(path_source_files, f)).st_size)
# Excluding files that are relatively small
thresh = np.median(sizes)/4
print(f'\nExcluding pickle files of size less than {thresh/1e6:0.3f} MB.')
filenames = [fn for fn, sz in zip(filenames, sizes) if sz>=thresh]

# Report filenames
path_source_files = os.path.abspath(path_source_files)
print(f'\nFILES IN THE SOURCE DIRECTORY: {path_source_files}')
pprint(filenames)

ModuleNotFoundError: No module named 'numpy'

In [ ]:
import datetime

In [ ]:
dd=filenames[4].split('_')[2]
datetime.date(int(dd[0]+dd[1])+2000,int(dd[2]+dd[3]),int(dd[4]+dd[5]))

In [ ]:
tm=[0,1,5]

In [ ]:
adj_aug=np.zeros([len(tm),995,995])
for i in range(len(tm)):
    with open(filenames[tm[i]], 'rb') as f:
        data = pkl.load(f)
        well = 'well003'
    adj_matrix = data[well]['adj_matrix_predicted']
    l=adj_matrix.shape[0]
    print( [filenames[tm[i]],l] )

In [ ]:
## add zeros to the adj mat such that they have the same dims 
adj_aug=np.zeros([len(tm),995,995])
for i in range(len(tm)):
    with open(filenames[tm[i]], 'rb') as f:
        data = pkl.load(f)
        well = 'well003'
    print(filenames[tm[i]])
    adj_matrix = data[well]['adj_matrix_predicted']
    votes = data[well]['votes']
    corr_peaks = data[well]['corr_peaks']
    synced_matrix = np.full(adj_matrix.shape, False)
    for key in corr_peaks.keys():
        if np.all(np.abs(np.array(corr_peaks[key]['delays'])) < data['parameters']['time_resolution']):
            synced_matrix[key[0], key[1]] = True
            synced_matrix[key[1], key[0]] = True
    filtered_matrix = np.logical_and(data[well]['adj_matrix_predicted'], np.logical_not(synced_matrix))
    l=filtered_matrix.shape[0]
    adj_aug[i][:l,:l]=filtered_matrix
    #adj_aug[i][:l,:l]=adj_matrix

In [ ]:
sum(adj_aug[0]>1)

In [ ]:
sum(np.transpose(adj_aug[0])-adj_aug[0])

In [ ]:
plt.imshow(adj_aug[0], interpolation='nearest')

In [ ]:
sum(sum(adj_aug[1]>0))

In [ ]:
from scipy.optimize import quadratic_assignment
from numpy import linalg as LA

In [ ]:
adj_gm=np.zeros([len(tm),995,995])
adj_gm[0]=adj_aug[0] ## use the first graph as baseline
for i in range(1, len(tm)):
    res = quadratic_assignment(adj_gm[i-1],adj_aug[i],options = {'maximize': True})
    perm = res['col_ind']
    adj_gm[i]=adj_aug[i][perm][:,perm]
    print([LA.norm(adj_gm[i]-adj_gm[i-1],'fro'),LA.norm(adj_aug[i]-adj_gm[i-1],'fro')]) ## make sure the difference between graphs are decreasing

In [ ]:
plt.imshow(adj_gm[3], interpolation='nearest')

In [ ]:
## product all adj mat to get common edges 
prod=adj_gm[0]
for i in range(1,len(tm)):
    prod=prod * adj_gm[i]

In [ ]:
from graspologic.datasets import load_drosophila_right
from graspologic.plot import heatmap
from graspologic.utils import binarize, symmetrize
import graspologic.utils as graspologic_utils

In [ ]:
## find the connected component for the common edge subgraph 
[a,ind]=graspologic_utils.largest_connected_component(prod,return_inds=True)
len(ind)

In [ ]:
# double check every adj mat is connected 
glist=np.zeros([len(tm),len(ind),len(ind)])
for i in range(len(tm)):
    glist[i]=adj_gm[i][ind][:,ind]
    print(graspologic_utils.is_fully_connected(glist[i]))

In [ ]:
from graspologic.embed import AdjacencySpectralEmbed

In [ ]:
n=len(ind)
## scree plot for one graph 
u,s,v=np.linalg.svd(glist[1], full_matrices=True)
plt.scatter(range(n),s)

In [ ]:
ed=1
Xhat_list=np.zeros([len(tm),n,2*ed])
for i in range(len(tm)):
    ase = AdjacencySpectralEmbed(n_components=ed)
    Xhat, Yhat = ase.fit_transform(glist[i])
    Xhat_list[i][:,list(np.arange(ed))]=Xhat
    Xhat_list[i][:,list(np.arange(ed,2*ed))]=Yhat

In [ ]:
from scipy.linalg import orthogonal_procrustes
D=np.zeros([len(tm),len(tm)])

In [ ]:
for i in range(len(tm)):
    for j in range(len(tm)):
        R, sca = orthogonal_procrustes(Xhat_list[i] , Xhat_list[j] )
        D[i,j]=LA.norm(Xhat_list[i] @ R -  Xhat_list[j],2)**2/n  

In [ ]:
D

In [ ]:
J=np.ones([len(tm),len(tm)])/len(tm)
I=np.eye(len(tm))
P=I-J